In [ ]:
from numpy import array
import pandas as pd
import numpy as np
import seaborn as sns
from time import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss, confusion_matrix, classification_report, roc_curve, auc
from sklearn.feature_extraction.text import TfidfTransformer
import pickle
from sklearn.utils import resample
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
data_type = 'Long' # All, Short, Med,Long
if data_type == 'All':
  df_reviews = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Original_data/IMDB_Review_w_Clean.json')
  df_kfold = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Original_data/StratifiedKfold_index.json')
else:
  df_reviews = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Original_data/'+data_type.lower()+'_clean_data.json')
  df_kfold = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Original_data/StratifiedKfold('+data_type.lower()+')_index.json')

In [ ]:
index_of_kfold = 0 # 0-4
df_train = df_reviews[['clean_review_text','is_spoiler']].iloc[df_kfold["train_index"][index_of_kfold]]
X_test =  df_reviews[['clean_review_text','is_spoiler']].iloc[df_kfold["test_index"][index_of_kfold]]
y_test =  df_reviews["is_spoiler"].iloc[df_kfold["test_index"][index_of_kfold]]
print(df_train.groupby('is_spoiler').count())

            clean_review_text
is_spoiler                   
False                   45497
True                    33582


In [ ]:
for i in range(0,5):
  index_of_kfold = i # 0-4
  df_train = df_reviews[['clean_review_text','is_spoiler']].iloc[df_kfold["train_index"][index_of_kfold]]
  X_test =  df_reviews[['clean_review_text','is_spoiler']].iloc[df_kfold["test_index"][index_of_kfold]]
  y_test =  df_reviews["is_spoiler"].iloc[df_kfold["test_index"][index_of_kfold]]

  df_X = df_train[["clean_review_text"]]
  df_Y = df_reviews[["is_spoiler"]]

  y_train = df_train.is_spoiler
  X_train = df_train.drop('is_spoiler', axis=1)

  # Import CountVectorizer class. 
  # CountVectorizer converts text data to matrix of token counts.
  # from sklearn.feature_extraction.text import CountVectorizer
  vectorizer = CountVectorizer(max_features = 20000, analyzer = 'word', 
                             dtype = np.float32, ngram_range=(1,1))

  # fit_transform() creates dictionary and return term-document matrix.
  X_train_counts = vectorizer.fit_transform(X_train['clean_review_text'].astype('U'))

  # Import TfidfTransformer class.
  # TfidfTransformer transoforms count matrix to tf-idf representation.
  tfidf_transformer = TfidfTransformer()

  # fit_transform transforms count matrix to tf-idf representation(vector).
  X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
  X_test_tfidf = vectorizer.transform(X_test['clean_review_text'].astype('U'))

  #Applying Naive Bayes
  t = time()
  clf_mnb = MultinomialNB()
  clf_mnb.fit(X_train_tfidf, y_train)
  y_pred = clf_mnb.predict(X_test_tfidf)
  print("====================Train=============================")
  #Calculate accuracy score
  mnb_score = accuracy_score(y_test, y_pred)
  auc_score = roc_auc_score(y_test,y_pred)
  print(mnb_score, auc_score)
  print(classification_report(y_test, y_pred))
  print(confusion_matrix(y_test, y_pred))
  print("====================Test=============================")
  mnb_score = accuracy_score(y_test, y_pred)
  auc_score = roc_auc_score(y_test, y_pred)
  print(mnb_score, auc_score)
  print(classification_report(y_test, y_pred))
  print(confusion_matrix(y_test, y_pred))

  df_evaluation  = pd.DataFrame(index=X_test.index)
  df_evaluation['nb(no_sample,tfidf)'] = y_pred
  df_evaluation.to_csv("/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Test_data/"+data_type+"_Data/Sample"+str(index_of_kfold)+"/nb(no_sample,tfidf).csv")

  filename = '/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Finished_model/'+data_type+'_Data/Sample'+str(index_of_kfold)+'/nb(no_sample,tfidf)_sample'+str(index_of_kfold)+'.sav'
  pickle.dump(clf_mnb, open(filename, 'wb'))

  # load the model from disk
  loaded_model = pickle.load(open(filename, 'rb'))
  result = loaded_model.score(X_test_tfidf, y_test)
  print(result)

====================Train=============================
0.6511886697015681 0.6309439911810433
              precision    recall  f1-score   support

       False       0.67      0.77      0.72     11374
        True       0.61      0.50      0.55      8396

    accuracy                           0.65     19770
   macro avg       0.64      0.63      0.63     19770
weighted avg       0.65      0.65      0.64     19770

[[8705 2669]
 [4227 4169]]
====================Test=============================
0.6511886697015681 0.6309439911810433
              precision    recall  f1-score   support

       False       0.67      0.77      0.72     11374
        True       0.61      0.50      0.55      8396

    accuracy                           0.65     19770
   macro avg       0.64      0.63      0.63     19770
weighted avg       0.65      0.65      0.64     19770

[[8705 2669]
 [4227 4169]]
0.6511886697015681
====================Train=============================
0.6572584724329793 0.644327154959